In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from scipy.stats import norm, expon, uniform, pareto
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv("Train_data.csv")
df.head()

In [38]:
original_copy = pd.read_csv("Train_data.csv")
data = original_copy
class_column = 'class'

#Exluding the class column
without_column = df.drop(columns='class')

#splitting data
train_data, test_data = train_test_split(without_column, test_size=0.3, random_state=42)

# Save the splits if needed
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

print("Data is prepared")

Data is prepared


In [ ]:
n_columns = train_data.select_dtypes(include=['number'])
z_scores = (n_columns - n_columns.mean()) / n_columns.std()

def detect_a(z_scores, threshold=3):

    anomalies = (np.abs(z_scores) > threshold).any(axis=1)
    return pd.DataFrame({'is_anomaly': anomalies})

# Example: Use a threshold of 3
threshold = 3
anomaly_results = detect_a(z_scores, threshold)

# Add 'is_anomaly' to the original training dataset
train_data['is_anomaly'] = anomaly_results['is_anomaly']

#Testing
thresholds = [1.5, 2.5, 3, 4.5]
for t in thresholds:
    anomalies = detect_a(z_scores, t)
    print(f"Threshold: {t}, Anomalies Detected: {anomalies['is_anomaly'].sum()}")

print("Z-score computation and anomaly detection successful!")

In [49]:
original_data = pd.read_csv("Train_data.csv")
train_data = pd.read_csv("Train_data.csv")

# Bring the 'class' column back from original data
train_data['class'] = original_data.loc[train_data.index, 'class']

# Convert 'class' column to binary values
train_data['class'] = train_data['class'].map({'anomaly': 1, 'normal': 0})

# Exclude columns with zero variance
numerical_columns = train_data.select_dtypes(include=['float64', 'int64']).columns
numerical_columns = [col for col in numerical_columns if train_data[col].std() > 0]


# Replace NaN/Inf values
train_data = train_data.replace([np.inf, -np.inf], np.nan).dropna()


# Candidate distributions to test
distributions = [
    stats.uniform, stats.expon, stats.norm, stats.pareto, stats.gamma, stats.lognorm
]

# Function to fit PDFs and calculate MSE
def fit_pdfs(column_data):
    range_min = column_data.min() - 0.1 * abs(column_data.min())
    range_max = column_data.max() + 0.1 * abs(column_data.max())
    x = np.linspace(range_min, range_max, 100)  # Adjusted range for fitting
    
    results = []
    for dist in distributions:
        try:
            params = dist.fit(column_data)
            pdf = dist.pdf(x, *params)
            mse = np.mean((np.histogram(column_data, bins=30, density=True)[0] - pdf[:30])**2)
            results.append((dist.name, mse, params, pdf))
        except Exception:
            pass  # Skip distributions that fail
    return sorted(results, key=lambda x: x[1])  # Sort by MSE

# Function to process and visualize multiple columns
def process_columns(data, numerical_columns):
    for column in numerical_columns:
        column_data = data[column]
        
        # Fit PDFs for entire data
        fit_results = fit_pdfs(column_data)
        best_fit = fit_results[0]
        
        print(f"Column: {column}")
        print(f"  Best fit: {best_fit[0]} with MSE = {best_fit[1]:.4f}")
        
        # Fit PDFs conditioned on anomalies
        anomaly_data = data[data["anomaly"] == 1][column]
        normal_data = data[data["anomaly"] == 0][column]
        
        anomaly_fit = fit_pdfs(anomaly_data)
        normal_fit = fit_pdfs(normal_data)
        
        print(f"  Best fit for anomalies: {anomaly_fit[0][0]} with MSE = {anomaly_fit[0][1]:.4f}")
        print(f"  Best fit for normals: {normal_fit[0][0]} with MSE = {normal_fit[0][1]:.4f}")
        
        # Plot data histogram and best fits
        x = np.linspace(column_data.min(), column_data.max(), 100)
        plt.hist(column_data, bins=30, density=True, alpha=0.5, label="Data Histogram")
        plt.plot(x, best_fit[3], label=f"Best Fit: {best_fit[0]} (MSE={best_fit[1]:.4f})", color="red")
        plt.title(f"Best PDF Fit for {column}")
        plt.xlabel("Value")
        plt.ylabel("Density")
        plt.legend()
        plt.show()



KeyError: 'class'